In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error
import xgboost as xgb

import json
import os

os.chdir("Test Data/")

random_state = 1

f2_1000_5train = pd.read_csv("Friedman2Sim/1000_5train.csv")
f2_5000_5train = pd.read_csv("Friedman2Sim/5000_5train.csv")
f2_1000_20train = pd.read_csv("Friedman2Sim/1000_20train.csv")
f2_5000_20train = pd.read_csv("Friedman2Sim/5000_20train.csv")
f2_1000_5test = pd.read_csv("Friedman2Sim/1000_5test.csv")
f2_5000_5test = pd.read_csv("Friedman2Sim/5000_5test.csv")
f2_1000_20test = pd.read_csv("Friedman2Sim/1000_20test.csv")
f2_5000_20test = pd.read_csv("Friedman2Sim/5000_20test.csv")

f3_1000_5train = pd.read_csv("Friedman3Sim/1000_5train.csv")
f3_5000_5train = pd.read_csv("Friedman3Sim/5000_5train.csv")
f3_1000_20train = pd.read_csv("Friedman3Sim/1000_20train.csv")
f3_5000_20train = pd.read_csv("Friedman3Sim/5000_20train.csv")
f3_1000_5test = pd.read_csv("Friedman3Sim/1000_5test.csv")
f3_5000_5test = pd.read_csv("Friedman3Sim/5000_5test.csv")
f3_1000_20test = pd.read_csv("Friedman3Sim/1000_20test.csv")
f3_5000_20test = pd.read_csv("Friedman3Sim/5000_20test.csv")


In [ ]:
def test_function(results_df, train, test, model, model_name, dataset_name):
    X_train = train.drop(['y'], axis = 1)
    y_train = train.y
    X_test = test.drop(['y'], axis = 1)
    y_test = test.y
    
    start_time = time.time()
    model.fit(X_train, y_train)
    mse = mean_squared_error(y_test, model.predict(X_test))
    total_time = (time.time()-start_time)
    results_df.loc[len(results_df)] = [dataset_name, model_name, mse, total_time]
    
    return results_df


In [ ]:
import time
def train_hyperparameters(mod, grid, output_name, train):
    start_time = time.time()
    X_train = train.drop(['y'], axis = 1)
    y_train = train.y
    
    cv = KFold(n_splits=5, shuffle=True, random_state=1)

    grid_search = GridSearchCV(estimator=mod, param_grid=grid, 
                               cv=cv, n_jobs=-1, verbose=1, scoring = 'neg_mean_squared_error')
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    print("Best score:", -1*grid_search.best_score_)
    print("Best params: ", best_params)
    print("Time: ", time.time()-start_time)

    with open('Hyperparameters/Friedman2&3Sim'+output_name+'.json', 'w') as f:
        json.dump(best_params, f)

# Random Forest

In [ ]:
# sqrt_features = int((airfoil_train_1.shape[1]-1)**0.5)

param_grid = {
    'n_estimators': [300],
    'max_depth': [12, 15, 18, 21, 24, 27, 30],
    'max_features': ["sqrt", "log2", 1.0, 0.5],
}

rf = RandomForestRegressor(random_state = 1)
train_hyperparameters(rf, param_grid, "rf_f2_1000_5", f2_1000_5train)
train_hyperparameters(rf, param_grid, "rf_f2_5000_5", f2_5000_5train)
train_hyperparameters(rf, param_grid, "rf_f2_1000_20", f2_1000_20train)
train_hyperparameters(rf, param_grid, "rf_f2_5000_20", f2_5000_20train)
train_hyperparameters(rf, param_grid, "rf_f3_1000_5", f3_1000_5train)
train_hyperparameters(rf, param_grid, "rf_f3_5000_5", f3_5000_5train)
train_hyperparameters(rf, param_grid, "rf_f3_1000_20", f3_1000_20train)
train_hyperparameters(rf, param_grid, "rf_f3_5000_20", f3_5000_20train)



## Testing random forest

In [ ]:
def rf_test_function(results_df, train, test, dataset_name, parameters_name):
    with open('Hyperparameters/Friedman2&3Sim/' + parameters_name + '.json') as f:
        best_params = json.load(f)
    best_rf = RandomForestRegressor(**best_params, random_state = 1)
    return test_function(results_df, train, test, best_rf, "Random Forest", dataset_name)



In [ ]:
results_df = pd.DataFrame(columns=['Dataset', 'Model', 'Model MSE', 'Time'])
results_df = rf_test_function(results_df, f2_1000_5train, f2_1000_5test, "f2_1000_5train", "rf_f2_1000_5")
results_df = rf_test_function(results_df, f2_5000_5train, f2_5000_5test, "f2_5000_5train", "rf_f2_5000_5")
results_df = rf_test_function(results_df, f2_1000_20train, f2_1000_20test, "f2_1000_20train", "rf_f2_1000_20")
results_df = rf_test_function(results_df, f2_5000_20train, f2_5000_20test, "f2_5000_20train", "rf_f2_5000_20")
results_df = rf_test_function(results_df, f3_1000_5train, f3_1000_5test, "f3_1000_5train", "rf_f3_1000_5")
results_df = rf_test_function(results_df, f3_5000_5train, f3_5000_5test, "f3_5000_5train", "rf_f3_5000_5")
results_df = rf_test_function(results_df, f3_1000_20train, f3_1000_20test, "f3_1000_20train", "rf_f3_1000_20")
results_df = rf_test_function(results_df, f3_5000_20train, f3_5000_20test, "f3_5000_20train", "rf_f3_5000_20")


In [ ]:
results_df.to_csv("RF Friedman2&3Sim Results.csv")
results_df


# XGBoost

In [ ]:
param_grid = {
    'max_depth': [4,6,8],
    'learning_rate': [0.01, 0.05, 0.1, 0.3],
    'reg_lambda':[0.1, 1, 10],
    'n_estimators':[100, 500, 1000],
#     'gamma': [0, 10, 100],
    'subsample': [0.6, 0.8, 1.0],
#     'colsample_bytree': [0.6, 0.8, 1.0]
}

xgboost = xgb.XGBRegressor(random_state=1)
train_hyperparameters(xgboost, param_grid, "xgb_f2_1000_5", f2_1000_5train)
train_hyperparameters(xgboost, param_grid, "xgb_f2_5000_5", f2_5000_5train)
train_hyperparameters(xgboost, param_grid, "xgb_f2_1000_20", f2_1000_20train)
train_hyperparameters(xgboost, param_grid, "xgb_f2_5000_20", f2_5000_20train)
train_hyperparameters(xgboost, param_grid, "xgb_f3_1000_5", f3_1000_5train)
train_hyperparameters(xgboost, param_grid, "xgb_f3_5000_5", f3_5000_5train)
train_hyperparameters(xgboost, param_grid, "xgb_f3_1000_20", f3_1000_20train)
train_hyperparameters(xgboost, param_grid, "xgb_f3_5000_20", f3_5000_20train)



In [ ]:
def xgb_test_function(results_df, train, test, dataset_name, parameters_name):
    with open('Hyperparameters/Friedman2&3Sim/' + parameters_name + '.json') as f:
        best_params = json.load(f)
    best_rf = xgb.XGBRegressor(**best_params, random_state = 1)
    return test_function(results_df, train, test, best_rf, "Xtreme Gradient Boosting", dataset_name)



In [ ]:
results_df = pd.DataFrame(columns=['Dataset', 'Model', 'Model MSE', 'Time'])
results_df = xgb_test_function(results_df, f2_1000_5train, f2_1000_5test, "f2_1000_5", "xgb_f2_1000_5")
results_df = xgb_test_function(results_df, f2_5000_5train, f2_5000_5test, "f2_5000_5", "xgb_f2_5000_5")
results_df = xgb_test_function(results_df, f2_1000_20train, f2_1000_20test, "f2_1000_20", "xgb_f2_1000_20")
results_df = xgb_test_function(results_df, f2_5000_20train, f2_5000_20test, "f2_5000_20", "xgb_f2_5000_20")
results_df = xgb_test_function(results_df, f3_1000_5train, f3_1000_5test, "f3_1000_5", "xgb_f3_1000_5")
results_df = xgb_test_function(results_df, f3_5000_5train, f3_5000_5test, "f3_5000_5", "xgb_f3_5000_5")
results_df = xgb_test_function(results_df, f3_1000_20train, f3_1000_20test, "f3_1000_20", "xgb_f3_1000_20")
results_df = xgb_test_function(results_df, f3_5000_20train, f3_5000_20test, "f3_5000_20", "xgb_f3_5000_20")

In [ ]:
results_df.to_csv("XGB Friedman2&3Sim Results.csv")
results_df
